#Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Import packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Paths setting

In [3]:
image_path='/content/gdrive/MyDrive/Personalized_Perception/Experiment_dataset/Cat_dataset/'

#Data Loading

##Loading rating sample experiment participant behavior data

In [4]:
rating_sample_data=np.load('/content/gdrive/MyDrive/Personalized_Perception/Observer experiment result/experiment result/Rating sampled experiment data/Rating sample 2000 result.npy',allow_pickle=True).item()

In [5]:
rating_sample_data.keys()

dict_keys(['Eddy', 'Jack Lo'])

In [6]:
participant='Jack Lo'

In [7]:
classes_distribution={}
class_imgs={}
for img,value in rating_sample_data[participant].items():
  if str(value) not in classes_distribution.keys():
    classes_distribution[f'{value}']=1
    class_imgs[f'{value}']=[img]
  else:
    classes_distribution[f'{value}']+=1
    class_imgs[f'{value}'].append(img)

In [8]:
classes_distribution

{'1.0': 202,
 '2.0': 407,
 '3.0': 487,
 '4.0': 354,
 '5.0': 313,
 '6.0': 158,
 '7.0': 79}

In [9]:
class_1=[]
class_2=[]
class_3=[]

for i in range(300):
  if class_imgs['1.0']:
    class_1.append(class_imgs['1.0'].pop())
  elif class_imgs['2.0']:
    class_2.append(class_imgs['2.0'].pop())
  else:
    class_3.append(class_imgs['3.0'].pop())

class_7=[]
class_6=[]
class_5=[]

for i in range(300):
  if class_imgs['7.0']:
    class_7.append(class_imgs['7.0'].pop())
  elif class_imgs['6.0']:
    class_6.append(class_imgs['6.0'].pop())
  else:
    class_5.append(class_imgs['5.0'].pop())

In [10]:
not_cute_data_train=class_1[:int(len(class_1)*0.8)]+class_2[:int(len(class_2)*0.8)]+class_3[:int(len(class_3)*0.8)]
cute_data_train=class_7[:int(len(class_7)*0.8)]+class_6[:int(len(class_6)*0.8)]+class_5[:int(len(class_5)*0.8)]
not_cute_data_test=class_1[int(len(class_1)*0.8):]+class_2[int(len(class_2)*0.8):]+class_3[int(len(class_3)*0.8):]
cute_data_test=class_7[int(len(class_7)*0.8):]+class_6[int(len(class_6)*0.8):]+class_5[int(len(class_5)*0.8):]

In [11]:
X_train_data=not_cute_data_train+cute_data_train
Y_train=[1 if i//len(not_cute_data_train) else 0 for i in range(len(cute_data_train)*2)]
X_test_data=not_cute_data_test+cute_data_test
Y_test=[1 if i//len(not_cute_data_test) else 0 for i in range(len(cute_data_test)*2)]

In [12]:
#Loading rating W latent space map
w_latent_space=np.load(r'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Image latent space data/W/rating_experiment_2000_image_W_latent_space_data_map.npy',allow_pickle=True).item()

In [13]:
X_train=[]
for img in X_train_data:
  X_train.append(w_latent_space[img[7:]][0,0,:])

X_test=[]
for img in X_test_data:
  X_test.append(w_latent_space[img[7:]][0,0,:])

In [14]:
X_train=np.array(X_train)
Y_train=np.array(Y_train)
X_test=np.array(X_test)
Y_test=np.array(Y_test)

#Loading linear svm from sklearn 



##Loading packages

In [15]:
from sklearn import svm
from sklearn import metrics

##set up functions and classification model

In [16]:
def getScores(estimator, x, y):
    yPred = estimator.predict(x)
    return (metrics.accuracy_score(y, yPred), 
         metrics.precision_score(y, yPred), 
         metrics.recall_score(y, yPred),
         metrics.f1_score(y, yPred))

def my_scorer(estimator, x, y):
    a, p, r ,f= getScores(estimator, x, y)
    print( a, p, r, f)
    return a,p,r,f

In [17]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train,Y_train)
y_pred=clf.predict(X_test)
table={'clf_name':'linear_svm'}
table['Accuracy']=metrics.accuracy_score(Y_test, y_pred)
table['Percision']=metrics.precision_score(Y_test, y_pred)
table['Recall']=metrics.recall_score(Y_test, y_pred)
table['F1_Score']=metrics.f1_score(Y_test, y_pred)

In [18]:
table

{'Accuracy': 0.7049180327868853,
 'F1_Score': 0.7096774193548387,
 'Percision': 0.6984126984126984,
 'Recall': 0.7213114754098361,
 'clf_name': 'linear_svm'}

##Save SVM model

In [19]:
import os
import joblib

In [ ]:
os.mkdir(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/{participant}/')
joblib.dump(clf,f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/{participant}/{participant}_rating_experiment.joblib')

##Load SVM model

In [21]:
clf=joblib.load(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/{participant}/{participant}_rating_experiment.joblib') 

#Regenerate image from StyleGan2 ADA

##Load StyleGAN-ada Genertor

Download StyleGAN-ada afhqcat model

In [22]:
%tensorflow_version 1.x
# ! pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html 

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.


In [23]:
import os
os.chdir('/content')
CODE_DIR = 'stylegan2-ada'

!git clone https://github.com/NVlabs/stylegan2-ada.git $CODE_DIR

Cloning into 'stylegan2-ada'...
remote: Enumerating objects: 74, done.
remote: Total 74 (delta 0), reused 0 (delta 0), pack-reused 74
Unpacking objects: 100% (74/74), done.


In [24]:
%cd /content/stylegan2-ada
import argparse
import os
import pickle
import re

import numpy as np
import PIL.Image

import dnnlib
import dnnlib.tflib as tflib
tflib.init_tf()
print('Loading networks from "%s"...' % 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/afhqcat.pkl')
with dnnlib.util.open_url('https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/afhqcat.pkl') as fp:
        _G, _D, Gs = pickle.load(fp)

/content/stylegan2-ada
Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/afhqcat.pkl"...


##Function for style_morphing

In [25]:
from PIL import Image

In [26]:
def reconstruct_latent_space(latent):
  reconstructed_latent=np.zeros((1,18,512))
  for i in range(len(reconstructed_latent[0])):
      reconstructed_latent[0][i]=latent
  return reconstructed_latent


In [52]:
def style_morphing(initial_img_latent_space, style_dir, steps, make_cuter):
  """
  Input:
    initial_img_latent_space -> numpy array [512,] shape: the image's W latent space from pSp encoder
    style_dir -> numpy array [512,] shape: the reconstructed SVM coef
    steps -> int: the morphing steps
    make_cuter -> bool: make the image cuter or uglier
  Output:
    morghing_imgs -> numpy array [steps,512,512,3] shape: regenerated image matrix
  """
  if make_cuter:
    target_latent_space=reconstruct_latent_space(style_dir)+reconstruct_latent_space(initial_img_latent_space)
  else:
    target_latent_space=reconstruct_latent_space(-style_dir)+reconstruct_latent_space(initial_img_latent_space)

  
  latent_morphing=np.linspace(initial_img_latent_space, target_latent_space, num=steps)
  # print(latent_morphing.shape)
  morghing_imgs=[]
  for latent_space in latent_morphing:
    morghing_imgs.append(Gs.components.synthesis.run(latent_space, output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:])
  return np.array(morghing_imgs)

##Define style_dir of cuteness

In [28]:
style_dir=clf.coef_[0]

##Evaluate correct_predicted sample

###Fetch correct_predicted Data

In [29]:
cor_predict_img=X_test[[Y_test==y_pred]]
cor_predict_label=Y_test[Y_test==y_pred]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [30]:
print(cor_predict_img.shape)
print(cor_predict_label.shape)

(86, 512)
(86,)


###plotting image



In [46]:
def get_concat_vertical(ims): 
  dst = Image.new('RGB', (ims[0].width, ims[0].height*len(ims)))
  print(len(ims))
  for i,img in enumerate(ims):

    dst.paste(img, (0, img.height*i))

  return dst

####Fetch cute Data

In [33]:
#randomly select five sample from cute data
np.random.seed(2000)
cute_mask=np.ones(cor_predict_label.shape)
cute_imgs=cor_predict_img[cor_predict_label==cute_mask]
cute_d=cor_predict_d[cor_predict_label==cute_mask]
randon_idx=np.random.choice([i for i in range(len(cute_imgs))],size=5,replace=False)

cute_sample=cute_imgs[randon_idx]
cute_sample_d=cute_d[randon_idx]

####Generate and save the imgs

In [53]:
#cuter style change
result=[]
result_images=[]
for latent,distance in zip(cute_sample,cute_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=True)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
    result_images.append(morpging_imgs[i])
  result.append(Image.fromarray(image_list))

In [54]:
result[0]

Output hidden; open in https://colab.research.google.com to view.

In [47]:
ims=[]
for i in range(0,5,1):
  ims.append(Image.fromarray(result_images[i]))
dst=get_concat_vertical(ims)

5


In [ ]:
dst

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_cuter/correct_perdicted_cute_sample_make_cuter_{i}_normalized.png')

In [55]:
#uglyer style change
result=[]
result_images=[]
for latent,distance in zip(cute_sample,cute_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=False)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
    result_images.append(morpging_imgs[i])
  result.append(Image.fromarray(image_list))

In [56]:
result[0]

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_uglier/correct_perdicted_cute_sample_make_uglyer_{i}_normalized.png')

####Fetch Ugly Data

In [ ]:
#randomly select five sample from ugly data
np.random.seed(2000)
ugly_mask=np.zeros(cor_predict_label.shape)
ugly_imgs=cor_predict_img[cor_predict_label==ugly_mask]
ugly_d=cor_predict_d[cor_predict_label==ugly_mask]
randon_idx=np.random.choice([i for i in range(len(ugly_imgs))],size=5,replace=False)

ugly_sample=ugly_imgs[randon_idx]
ugly_sample_d=ugly_d[randon_idx]

####Generate and save the imgs

In [ ]:
#cuter style change
result=[]
for latent,distance in zip(ugly_sample,ugly_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=True)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_cuter/correct_perdicted_ugly_sample_make_cuter_{i}_normalized.png')

In [ ]:
#uglyer style change
result=[]
for latent,distance in zip(ugly_sample,ugly_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=False)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_uglier/correct_perdicted_ugly_sample_make_uglyer_{i}_normalized.png')

##Evaluate mis_predicted sample

###Fetch mis_predicted Data

In [ ]:
mis_predict_img=X_test[[Y_test!=y_pred]]
mis_predict_label=Y_test[Y_test!=y_pred]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [ ]:
#get distance from dicision boundry for each sample
mis_predict_d=clf.decision_function(mis_predict_img)

###plotting image

####Fetch cute Data

In [ ]:
#randomly select five sample from cute data
np.random.seed(2000)
cute_mask=np.ones(mis_predict_label.shape)
cute_imgs=mis_predict_img[mis_predict_label==cute_mask]
cute_d=mis_predict_d[mis_predict_label==cute_mask]
randon_idx=np.random.choice([i for i in range(len(cute_imgs))],size=5,replace=False)

cute_sample=cute_imgs[randon_idx]
cute_sample_d=cute_d[randon_idx]

####Generate and save the imgs

In [ ]:
#cuter style change
result=[]
result_images=[]
for latent,distance in zip(cute_sample,cute_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=True)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
    result_images.append(morpging_imgs[i])
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_cuter/mis_perdicted_cute_sample_make_cuter_{i}_normalized.png')

In [ ]:
#uglyer style change
result=[]
for latent,distance in zip(cute_sample,cute_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=False)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_uglier/mis_perdicted_cute_sample_make_uglyer_{i}_normalized.png')

####Fetch ugly Data

In [ ]:
#randomly select five sample from ugly data
np.random.seed(2000)
ugly_mask=np.zeros(mis_predict_label.shape)
ugly_imgs=mis_predict_img[mis_predict_label==ugly_mask]
ugly_d=mis_predict_d[mis_predict_label==ugly_mask]
randon_idx=np.random.choice([i for i in range(len(ugly_imgs))],size=5,replace=False)

ugly_sample=ugly_imgs[randon_idx]
ugly_sample_d=ugly_d[randon_idx]

####Generate and save the imgs

In [ ]:
#cuter style change
result=[]
for latent,distance in zip(ugly_sample,ugly_sample_d):
  morpging_imgs=style_morphing(latent, style_dir/distance, distance, steps=5)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_cuter/mis_perdicted_ugly_sample_make_cuter_{i}.png')

In [ ]:
#uglyer style change
result=[]
for latent,distance in zip(ugly_sample,ugly_sample_d):
  morpging_imgs=style_morphing(latent, -style_dir/distance, distance, steps=5)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_uglier/mis_perdicted_ugly_sample_make_uglyer_{i}.png')

##Evaluate Neutral sample

###Fetch Neutral Data

In [ ]:
neutral_data=[]
for img in class_imgs['4.0']:
  neutral_data.append(w_latent_space[img[7:]][0,0])
neutral_data=np.array(neutral_data)

In [ ]:
neutral_data_d=clf.decision_function(neutral_data)

In [ ]:
neutral_pred=clf.predict(neutral_data)

####Fetch Samples predicted as Cute 

In [ ]:
#randomly select five sample from cute data
np.random.seed(2000)
cute_mask=np.ones(neutral_pred.shape)
pred_cute_data=neutral_data[neutral_pred==cute_mask]
pred_cute_data_d=neutral_data_d[neutral_pred==cute_mask]
randon_idx=np.random.choice([i for i in range(len(pred_cute_data))],size=5,replace=False)

pred_cute_sample=pred_cute_data[randon_idx]
pred_cute_sample_d=pred_cute_data_d[randon_idx]

####Generate and save the imgs

In [ ]:
#cuter style change
result=[]
result_images=[]
for latent,distance in zip(pred_cute_sample,pred_cute_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=True)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
    result_images.append(morpging_imgs[i])
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_cuter/perdicted_cute_neutral_sample_make_cuter_{i}_normalized.png')

In [ ]:
#uglier style change
result=[]
for latent,distance in zip(pred_cute_sample,pred_cute_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=False)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_uglier/perdicted_cute_neutral_sample_make_uglier_{i}_normalized.png')

####Fetch Samples predicted as ugly 

In [ ]:
#randomly select five sample from data
np.random.seed(2000)
ugly_mask=np.zeros(neutral_pred.shape)
pred_ugly_data=neutral_data[neutral_pred==ugly_mask]
pred_ugly_data_d=neutral_data_d[neutral_pred==ugly_mask]
randon_idx=np.random.choice([i for i in range(len(pred_ugly_data))],size=5,replace=False)

pred_ugly_sample=pred_ugly_data[randon_idx]
pred_ugly_sample_d=pred_ugly_data_d[randon_idx]

####Generate and save the imgs

In [ ]:
#cuter style change
result=[]
for latent,distance in zip(pred_ugly_sample,pred_ugly_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=True)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_cuter/perdicted_ugly_neutral_sample_make_cuter_{i}_normalized.png')

In [ ]:
#uglier style change
result=[]
for latent,distance in zip(pred_ugly_sample,pred_ugly_sample_d):
  morpging_imgs=style_morphing(latent, style_dir, steps=5, make_cuter=False)
  image_list=Gs.components.synthesis.run(reconstruct_latent_space(latent), output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True))[0,:]# add the reconstructed real image
  for i in range(len(morpging_imgs)):
    image_list=np.concatenate([image_list,morpging_imgs[i]], axis=1)
  result.append(Image.fromarray(image_list))

In [ ]:
for i,img in enumerate(result):
  img.save(f'/content/gdrive/MyDrive/Personalized_Perception/pSp_encoder_research_2022Spring/Machine Learning model experiment with W latent space/SVM/Eddy/Style_changed_image/make_uglier/perdicted_ugly_neutral_sample_make_uglier_{i}_normalized.png')